In [1]:
# Import necessary modules
import sys
sys.path.append("../src/confpickem/")
from yahoo_pickem_scraper import YahooPickEm, calculate_player_stats
from confidence_pickem_sim import ConfidencePickEmSimulator, Player
from yahoo_pickem_integration import convert_yahoo_to_simulator_format, convert_yahoo_picks_to_dataframe
import pandas as pd

In [2]:
# Pull data from Yahoo
week = 10
league_id = 6207
num_sims = 1000
# num_entries = 50

yahoo_data = YahooPickEm(week, league_id, cookies_file='../src/confpickem/cookies.txt')
# yahoo_data.games
# yahoo_data.players
# yahoo_data.results

In [3]:
# Initialize simulator instance and feed in NFL game details
simulator = ConfidencePickEmSimulator(num_sims=num_sims)
games_df = convert_yahoo_to_simulator_format(yahoo_data, ignore_results=True)
simulator.add_games_from_dataframe(games_df)

# games_df
# simulator.games

In [4]:
# Create average players and feed into simulator
# simulator.players = [
#         Player(
#             name=f"Player {ind + 1}",
#             skill_level=0.749,
#             crowd_following=0.737,
#             confidence_following=0.826
#         ) for ind in range(num_entries)
#     ]

# Calculate player statistics from previous results
stats = calculate_player_stats(
        league_id=6207,
        weeks=range(1, week),
        cookies_file='../src/confpickem/cookies.txt'
    )
stats['me'] = stats.player == "Firman's Educated Guesses"
stats = stats.sort_values(by="me",ascending=False,ignore_index=True)

player_stats = [
        Player(
            name=stats.iloc[ind].player,
            skill_level=stats.iloc[ind].skill_level,
            crowd_following=stats.iloc[ind].crowd_following,
            confidence_following=stats.iloc[ind].confidence_following
        ) for ind in range(stats.shape[0])
    ]
simulator.players = player_stats

# simulator.players

In [5]:
# Run simulation with random picks and outcomes
picks_df = simulator.simulate_picks()
outcomes = simulator.simulate_outcomes()
# picks_df.groupby(["player","game","picked_home"]).size().sort_values(ascending=False)
# outcomes.sum(axis=0)

# Analyze results
stats_orig = simulator.analyze_results(picks_df, outcomes)
for stat in stats_orig:
    stats_orig[stat] = stats_orig[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})

summary_orig = pd.merge(left=stats_orig["win_pct"],right=stats_orig["expected_points"],how="inner",on="player")
summary_orig = pd.merge(left=summary_orig,right=stats_orig["point_std"],how="inner",on="player")
summary_orig = pd.merge(left=summary_orig,right=stats_orig["value_at_risk"],how="inner",on="player")
summary_orig

,player,win_pct,expected_points,point_std,value_at_risk
0,#FlytheW,0.034,58.9205,15.557521,33.000
1,9 inches,0.017,55.3370,15.812177,29.500
2,Arcedinero,0.021,57.7570,15.419946,32.975
3,BFense,0.006,53.7200,15.771393,27.000
4,Ballbusters,0.009,54.3410,15.715365,28.950
5,Balz Deep,0.018,54.4915,16.234569,27.475
6,Big Daddy125,0.044,60.8240,16.275547,32.950
7,BigShroomSecurity,0.027,58.2835,15.369974,32.000
8,Boats n Hoes,0.014,54.5050,16.225173,27.500
9,Boldz36,0.015,55.0545,15.771002,28.000


In [6]:
# Identify optimal picks for the week
optimal_picks = simulator.optimize_picks()

Game(home_team='Den', away_team='KC', vegas_win_prob=0.8472, crowd_home_pick_pct=0.05, crowd_home_confidence=3.5, crowd_away_confidence=11.2, week=10, kickoff_time=Timestamp('2024-11-10 13:00:00-0518', tz='UTC-05:18'), actual_outcome=None, picks_locked=False)
68.3535
58.8545
{'Den': 14}
Game(home_team='Buf', away_team='Ind', vegas_win_prob=0.8224, crowd_home_pick_pct=0.97, crowd_home_confidence=10.4, crowd_away_confidence=4.1, week=10, kickoff_time=Timestamp('2024-11-10 13:00:00-0518', tz='UTC-05:18'), actual_outcome=None, picks_locked=False)
78.094
70.118
{'Den': 14, 'Buf': 13}
Game(home_team='Ten', away_team='LAC', vegas_win_prob=0.8162, crowd_home_pick_pct=0.02, crowd_home_confidence=4.9, crowd_away_confidence=10.2, week=10, kickoff_time=Timestamp('2024-11-10 16:05:00-0518', tz='UTC-05:18'), actual_outcome=None, picks_locked=False)
83.004
74.2685
{'Den': 14, 'Buf': 13, 'Ten': 12}
Game(home_team='Min', away_team='Jax', vegas_win_prob=0.8130999999999999, crowd_home_pick_pct=0.97, crow

In [7]:
# Setting my picks to the optimized values
mask = (picks_df.player == simulator.players[0].name)
for team, pts in optimal_picks.items():
    game_mask = picks_df[mask].game.str.contains(team)
    is_home = (picks_df.loc[mask & game_mask, 'game'].str.split('@').str[1] == team).astype(float)
    picks_df.loc[mask & game_mask, 'picked_home'] = is_home
    picks_df.loc[mask & game_mask, 'confidence'] = float(pts)

# Analyze results
stats_new = simulator.analyze_results(picks_df, outcomes)
for stat in stats_new:
    stats_new[stat] = stats_new[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
summary_new = pd.merge(left=stats_new["win_pct"],right=stats_new["expected_points"],how="inner",on="player")
summary_new = pd.merge(left=summary_new,right=stats_new["point_std"],how="inner",on="player")
summary_new = pd.merge(left=summary_new,right=stats_new["value_at_risk"],how="inner",on="player")
summary_new.sort_values(by="win_pct",ascending=False)


,player,win_pct,expected_points,point_std,value_at_risk
19,Firman's Educated Guesses,0.284,81.2130,12.781823,58.000
6,Big Daddy125,0.035,60.8240,16.275547,32.950
30,MDTwinTeam,0.034,61.2665,15.180256,36.000
25,John K,0.028,60.0410,15.683201,33.000
0,#FlytheW,0.026,58.9205,15.557521,33.000
12,Caking,0.026,60.7730,15.663742,34.000
32,Matt,0.025,57.3940,16.299876,31.000
16,Diddypicks,0.023,58.1285,16.458983,32.000
7,BigShroomSecurity,0.022,58.2835,15.369974,32.000
33,Mickey's Team,0.019,57.3570,15.732850,30.950


In [11]:
# Pulling actual picks from Yahoo
picks_df_actual = convert_yahoo_picks_to_dataframe(yahoo_data, num_sims)#, optimal_picks)
print(picks_df_actual.loc[picks_df_actual.player == "Firman's Educated Guesses",["game","picked_home"]].drop_duplicates())
# picks_df.picked_home.unique()

# Simulating win probabilities given actual picks
outcomes = simulator.simulate_outcomes()
stats_actual = simulator.analyze_results(picks_df_actual, outcomes)
for stat in stats_actual:
    stats_actual[stat] = stats_actual[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
summary_actual = pd.merge(left=stats_actual["win_pct"],right=stats_actual["expected_points"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["point_std"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["value_at_risk"],how="inner",on="player")
summary_actual.sort_values(by="win_pct",ascending=False)

           game  picked_home
711000  Bal@Cin        False
712000  Car@NYG         True
713000   Chi@NE        False
714000  Ind@Buf        False
715000   KC@Den        False
716000   NO@Atl        False
717000    TB@SF         True
718000  Was@Pit        False
719000  Jax@Min        False
720000  LAC@Ten         True
721000  Dal@Phi         True
722000  Ari@NYJ         True
723000  Hou@Det        False
724000  LAR@Mia         True


,player,win_pct,expected_points,point_std,value_at_risk
44,Rent’s Due Again,0.212,67.089,13.403908,43.00
49,Victorious Bastard,0.171,65.636,13.619067,41.00
45,ShowMeYourTDs,0.137,64.064,13.336858,41.00
19,Fishy Picks,0.063,63.424,13.120609,42.00
35,NoWayIGotMorePointsThenYou?!?!,0.058,59.236,13.299292,36.00
18,Firman's Educated Guesses,0.043,46.429,13.642033,24.00
15,Diddypicks,0.038,53.154,12.774250,31.00
29,MDTwinTeam,0.036,51.244,13.255949,31.00
14,DO YOU FEEL LUCKY PUNK,0.027,59.458,13.629527,36.00
40,Pikachu,0.025,45.348,13.853766,24.00


In [ ]:
# Identifying relative importance of each game
importance = simulator.assess_game_importance(picks_df_actual)
print("\nGame Importance Analysis:")
print(importance[["game","points_bid","win_probability","loss_probability","total_impact"]].sort_values('total_impact', ascending=False).to_string(index=False))


Game Importance Analysis:
       game  points_bid  win_probability  loss_probability  win_delta  \
3   Ind@Buf        12.0            0.146             0.008      0.111   
8   Jax@Min         3.0            0.083             0.025      0.048   
12  Hou@Det         5.0            0.072             0.015      0.037   
7   Was@Pit         8.0            0.071             0.022      0.036   
5    NO@Atl         4.0            0.076             0.028      0.041   
0   Bal@Cin        14.0            0.065             0.020      0.030   
6     TB@SF        11.0            0.047             0.004      0.012   
11  Ari@NYJ        10.0            0.053             0.010      0.018   
9   LAC@Ten         1.0            0.043             0.009      0.008   
1   Car@NYG        13.0            0.041             0.015      0.006   
13  LAR@Mia         2.0            0.039             0.018      0.004   
2    Chi@NE         7.0            0.040             0.033      0.005   
10  Dal@Phi         9.0 

In [10]:
# # Checking who actually won
# simulator = ConfidencePickEmSimulator(num_sims=num_sims)
# games_df = convert_yahoo_to_simulator_format(yahoo_data, ignore_results=False)
# simulator.add_games_from_dataframe(games_df)
# outcomes = simulator.simulate_outcomes()

# stats_actual = simulator.analyze_results(picks_df_actual, outcomes)
# for stat in stats_actual:
#     stats_actual[stat] = stats_actual[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
# summary_actual = pd.merge(left=stats_actual["win_pct"],right=stats_actual["expected_points"],how="inner",on="player")
# summary_actual = pd.merge(left=summary_actual,right=stats_actual["point_std"],how="inner",on="player")
# summary_actual = pd.merge(left=summary_actual,right=stats_actual["value_at_risk"],how="inner",on="player")
# summary_actual.sort_values(by=["win_pct","expected_points"],ascending=False)